**Heyy there! We're gonna do fine tuning of LDM for the operation 'image-to-text' in this colaboratory**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!apt-get update
!apt-get install build-essential libopenmpi-dev liblzma-dev zlib1g-dev

# Install PyTorch
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# Install ldm-finetune
!git clone https://github.com/laion-ai/ldm-finetune.git
!cd ldm-finetune
!pip install -e .
!pip install -r sample_data/requirements.txt

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
liblzma-dev is already the newest version (5.2.5-2ubuntu1).
libopenmpi-dev is

In [ ]:
!nvidia-smi

Tue Jul  2 19:54:50 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# OpenAI CLIP ViT-L/14
!wget -P /root/.cache/clip "https://openaipublic.azureedge.net/clip/models/b8cca3fd41ae0c99ba7e8951adf17d267cdb84cd88be6f7c2e0eca1737a03836/ViT-L-14.pt"

# jack000 - inpaint.pt
# !wget --continue https://dall-3.com/models/glid-3-xl/inpaint.pt


# conceptual captions - model.pt
!wget --continue '/content/gdrive/MyDrive/ldmfinetune/model.ckpt'


# BERT Text Encoder
!wget --continue https://dall-3.com/models/glid-3-xl/bert.pt

# kl-f8 VAE backbone
!wget --continue https://dall-3.com/models/glid-3-xl/kl-f8.pt

--2024-07-02 19:54:50--  https://openaipublic.azureedge.net/clip/models/b8cca3fd41ae0c99ba7e8951adf17d267cdb84cd88be6f7c2e0eca1737a03836/ViT-L-14.pt
Resolving openaipublic.azureedge.net (openaipublic.azureedge.net)... 13.107.253.69, 2620:1ec:29:1::69
Connecting to openaipublic.azureedge.net (openaipublic.azureedge.net)|13.107.253.69|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 932768134 (890M) [application/octet-stream]
Saving to: ‘/root/.cache/clip/ViT-L-14.pt’

ViT-L-14.pt         100%[===================>] 889.56M   127MB/s    in 7.7s    

2024-07-02 19:54:58 (116 MB/s) - ‘/root/.cache/clip/ViT-L-14.pt’ saved [932768134/932768134]

/content/gdrive/MyDrive/ldmfinetune/model.ckpt: Scheme missing.
--2024-07-02 19:54:58--  https://dall-3.com/models/glid-3-xl/bert.pt
Resolving dall-3.com (dall-3.com)... 104.21.74.163, 172.67.204.67, 2606:4700:3034::ac43:cc43, ...
Connecting to dall-3.com (dall-3.com)|104.21.74.163|:443... connected.
HTTP request sent, awaitin

In [ ]:
# images,captions
# "FLIR_00310_PreviewData.jpeg","a car going swiftly in the center of road in a low precipitation low wind high temperature dusty area"
# "FLIR_07908_PreviewData.jpeg","scene of a road people standing near their cars in a high wind low temperature little precipitation cloudy area"
# ////

In [ ]:
# !pip install encoders

In [ ]:
#!/bin/bash
# Finetune glid-3-xl inpaint.pt on your own webdataset.
# Note: like all one-off scripts, this is likely to become out of date at some point.
# running python scripts/image_train_inpaint.py --help will give you more info.

# !pip install einops
# !pip install blobfile
# !pip install blobfile
# !pip install mpi4py
# !pip install webdataset
# !pip install braceexpand
# !pip install -r requirements.txt

import argparse
import torch

# model flags
use_fp16=False # TODO can cause more trouble than it's worth.
MODEL_FLAGS="--dropout 0.1 --attention_resolutions 32,16,8 --class_cond False --diffusion_steps 1000 --image_size 32 --learn_sigma False --noise_schedule linear --num_channels 320 --num_heads 8 --num_res_blocks 2 --resblock_updown False --use_fp16 $use_fp16 --use_scale_shift_norm False"

# checkpoint flags
resume_checkpoint="/content/gdrive/MyDrive/ldmfinetune/model.ckpt"
kl_model="/content/kl-f8.pt"
bert_model="/content/bert.pt"

# training flags

epochs=80
shard_size=512
batch_size=1
microbatch=-1
lr=1e-6 # lr=1e-5 seems to be stable. going above 3e-5 is not stable.
ema_rate=0.9999 # TODO you may want to lower this to 0.999, 0.99, 0.95, etc.
random_crop=False
random_flip=False
cache_dir="cache"
image_key="jpg"
caption_key="txt"
data_dir="/content/dataset.csv" # TODO set this to a real path

# interval flags
sample_interval=100
log_interval=1
save_interval=2000

CKPT_FLAGS="--kl_model $kl_model --bert_model $bert_model --resume_checkpoint $resume_checkpoint"
INTERVAL_FLAGS="--sample_interval $sample_interval --log_interval $log_interval --save_interval $save_interval"
TRAIN_FLAGS="--epochs $epochs --shard_size $shard_size --batch_size $batch_size --microbatch $microbatch --lr $lr --random_crop $random_crop --random_flip $random_flip --cache_dir $cache_dir --image_key $image_key --caption_key $caption_key --data_dir $data_dir"
COMBINED_FLAGS="$MODEL_FLAGS $CKPT_FLAGS $TRAIN_FLAGS $INTERVAL_FLAGS"
!export OPENAI_LOGDIR=./erlich_on_pixel_logs_run6_part2/
!export TOKENIZERS_PARALLELISM=false

# TODO comment out a line below to train either on a single GPU or multi-GPU
# single GPU
!python /content/ldm-finetune/image_train_inpaint.py $COMBINED_FLAGS
# !tail -f /content/ldm-finetune/image_train_inpaint.log

# or multi-GPU
# mpirun -n 8 python scripts/image_train_inpaint.py $COMBINED_FLAGS

Logging to /tmp/openai-2024-07-02-20-27-58-458018
loading clip...
loading vae...
[rank0]: Traceback (most recent call last):
[rank0]:   File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 540, in _check_seekable
[rank0]:     f.seek(f.tell())
[rank0]: AttributeError: 'NoneType' object has no attribute 'seek'

[rank0]: During handling of the above exception, another exception occurred:

[rank0]: Traceback (most recent call last):
[rank0]:   File "/content/ldm-finetune/image_train_inpaint.py", line 254, in <module>
[rank0]:     main()
[rank0]:   File "/content/ldm-finetune/image_train_inpaint.py", line 48, in main
[rank0]:     encoder = torch.load(args.kl_model, map_location="cpu")
[rank0]:   File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 997, in load
[rank0]:     with _open_file_like(f, 'rb') as opened_file:
[rank0]:   File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 449, in _open_file_like
[rank0]:     retur

In [ ]:
# !wget https://huggingface.co/laion/ongo/resolve/main/ongo.pt